In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import sys

In [2]:
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

In [3]:
train = pd.read_csv('app_train_domain.csv')
test = pd.read_csv('app_test_domain.csv')

# 5 - Feature Engineering
----------

## 5.1 - Création de Feature métier 

Je commence par créer des variables métier en m'inspirant de ce script :  https://www.kaggle.com/jsaguiar/updated-0-792-lb-lightgbm-with-simple-features :
- CREDIT_INCOME_PERCENT : le pourcentage du montant du crédit par rapport au revenu du client (=Taux d'endettement).
- INCOME_CREDIT_PERC : le pourcentage du revenu du client sur le montant du crédit (=coefficient de remboursement)
- ANNUITY_INCOME_PERCENT : le pourcentage de l'annuité du prêt par rapport au revenu du client (= taux d'effort)
- CREDIT_TERM : mensualité / montant du prêt (=taux de remboursement ou coefficient d'amortissement)
- DAYS_EMPLOYED_PERCENT : le pourcentage des jours de travail par rapport à l'âge du client.
- INCOME_PER_PERSON : le revenu du client sur le nombre de membres de la famille

In [4]:
train['CREDIT_INCOME_PERCENT'] = train['AMT_CREDIT'] / train['AMT_INCOME_TOTAL']
train['INCOME_CREDIT_PERC'] = train['AMT_INCOME_TOTAL'] / train['AMT_CREDIT']
train['ANNUITY_INCOME_PERCENT'] = train['AMT_ANNUITY'] / train['AMT_INCOME_TOTAL']
train['CREDIT_TERM'] = train['AMT_ANNUITY'] / train['AMT_CREDIT']
train['DAYS_EMPLOYED_PERCENT'] = train['DAYS_EMPLOYED'] / train['DAYS_BIRTH']
train['INCOME_PER_PERSON'] = train['AMT_INCOME_TOTAL'] / train['CNT_FAM_MEMBERS']

In [5]:
test['CREDIT_INCOME_PERCENT'] = test['AMT_CREDIT'] / test['AMT_INCOME_TOTAL']
test['INCOME_CREDIT_PERC'] = test['AMT_INCOME_TOTAL'] / test['AMT_CREDIT']
test['ANNUITY_INCOME_PERCENT'] = test['AMT_ANNUITY'] / test['AMT_INCOME_TOTAL']
test['CREDIT_TERM'] = test['AMT_ANNUITY'] / test['AMT_CREDIT']
test['DAYS_EMPLOYED_PERCENT'] = test['DAYS_EMPLOYED'] / test['DAYS_BIRTH']
test['INCOME_PER_PERSON'] = test['AMT_INCOME_TOTAL'] / test['CNT_FAM_MEMBERS']

## 5.2 - Création de fonctions pour automatiser le Feature Engineering depuis les autres datasets  

Sources :
- https://www.kaggle.com/code/willkoehrsen/introduction-to-manual-feature-engineering
- https://www.kaggle.com/code/willkoehrsen/introduction-to-manual-feature-engineering-p2

### 5.2.1 - Fonction pour traiter les variables numériques

La fonction agg_numeric agrège les variables numériques d'un dataframe. Elle calcule des statistiques de base (comme la moyenne, le minimum, le maximum et la somme) pour chaque variable en question.

In [6]:
def agg_numeric(df, group_var, df_name):
    """Agrège les valeurs numériques dans un dataframe. Cela peut
    être utilisé pour créer des caractéristiques pour chaque instance de la variable de groupement.
    
    Paramètres
    --------
        df (dataframe): 
            le dataframe sur lequel calculer les statistiques
        group_var (string): 
            la variable selon laquelle grouper le df
        df_name (string): 
            la variable utilisée pour renommer les colonnes
        
    Retour
    --------
        agg (dataframe): 
            un dataframe avec les statistiques agrégées pour 
            toutes les colonnes numériques. Chaque instance de la variable de groupement aura 
            les statistiques (moyenne, min, max, somme; actuellement supportées) calculées. 
            Les colonnes sont également renommées pour suivre les caractéristiques créées.
    
    """
    # Supprimer les variables d'identification autres que la variable de groupement
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns=col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Grouper par la variable spécifiée et calculer les statistiques
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Besoin de créer de nouveaux noms de colonnes
    columns = [group_var]

    # Itérer à travers les noms des variables
    for var in agg.columns.levels[0]:
        # Ignorer la variable de groupement
        if var != group_var:
            # Itérer à travers les noms des statistiques
            for stat in agg.columns.levels[1][:-1]:
                # Créer un nouveau nom de colonne pour la variable et la statistique
                columns.append(f'{df_name}_{var}_{stat}')

    agg.columns = columns

    # Supprimer les colonnes avec toutes les valeurs redondantes
    _, idx = np.unique(agg, axis=1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg

### 5.2.2 - Fonction pour traiter les variables catégorielles

Cette fonction va gérer les variables catégorielles. Elle prendra la même forme que la fonction agg_numeric, c'est-à-dire qu'elle acceptera un dataframe et une variable de regroupement. Elle calculera ensuite les effectifs et les effectifs normalisés de chaque catégorie pour toutes les variables catégorielles de la base de données.

In [7]:
def agg_categorical(df, parent_var, df_name):
    """
    Agrège les caractéristiques catégorielles dans un dataframe enfant
    pour chaque observation de la variable parent.
    
    Paramètres
    --------
    df : dataframe 
        Le dataframe pour lequel calculer les décomptes de valeurs.
        
    parent_var : string
        La variable selon laquelle grouper et agréger le dataframe. Pour chaque
        valeur unique de cette variable, le dataframe final aura une ligne.
        
    df_name : string
        Variable ajoutée devant les noms de colonnes pour suivre les colonnes.

    Retour
    --------
    categorical : dataframe
        Un dataframe avec des statistiques agrégées pour chaque observation de `parent_var`
        Les colonnes sont également renommées et les colonnes avec des valeurs dupliquées sont supprimées.
    """
    
    # Sélectionner les colonnes catégorielles
    categorical = pd.get_dummies(df.select_dtypes(include=['object', 'category']))
    
    if categorical.empty:
        return pd.DataFrame({parent_var: df[parent_var]}).groupby(parent_var).size().reset_index(name='count')
    
    # S'assurer de mettre l'identifiant sur la colonne
    categorical[parent_var] = df[parent_var]

    # Grouper par la variable de groupe et calculer la somme et la moyenne
    categorical = categorical.groupby(parent_var).agg(['sum', 'count', 'mean'])
    
    noms_colonnes = []
    
    # Itérer à travers les colonnes du niveau 0
    for var in categorical.columns.levels[0]:
        # Itérer à travers les statistiques du niveau 1
        for stat in ['sum', 'count', 'mean']:
            # Créer un nouveau nom de colonne
            noms_colonnes.append(f'{df_name}_{var}_{stat}')
    
    categorical.columns = noms_colonnes
    
    # Supprimer les colonnes dupliquées par valeurs
    _, idx = np.unique(categorical, axis=1, return_index=True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

## 5.3 - Feature Engineering sur les dataframe 'bureau' et 'bureau_balance'

Nous disposons à présent de tous les éléments nécessaires pour intégrer les informations relatives aux prêts antérieurs contractés auprès d'autres institutions et les informations relatives aux paiements mensuels de ces prêts dans le dataframe principal.

POUR MEMOIRE :
* **bureau**: contient les données concernant les crédits antérieurs du client auprès d'autres institutions financières. Chaque crédit antérieur a sa propre ligne dans le bureau, mais un prêt dans les données de la demande peut avoir plusieurs crédits antérieurs.
* **bureau_balance**: données mensuelles sur les crédits antérieurs dans le bureau. Chaque ligne correspond à un mois d'un crédit antérieur, et un seul crédit antérieur peut avoir plusieurs lignes, une pour chaque mois de la durée du crédit. 

In [8]:
# Lecture des fichiers

bureau = pd.read_csv('bureaux.csv')
bureau_balance = pd.read_csv('bureau_balances.csv')

In [9]:
# Application des fonctions sur 'bureau'
bureau_counts = agg_categorical(bureau, parent_var='SK_ID_CURR', df_name='bureau')
bureau_agg = agg_numeric(bureau.drop(columns=['SK_ID_BUREAU']), group_var='SK_ID_CURR', df_name='bureau')


In [10]:
# Application des fonctions sur 'bureau_balance'
bureau_balance_counts = agg_categorical(bureau_balance, parent_var='SK_ID_BUREAU', df_name='bureau_balance')
bureau_balance_agg = agg_numeric(bureau_balance, group_var='SK_ID_BUREAU', df_name='bureau_balance')


In [11]:
# Agrégation des statistiques de 'bureau_balance' par client

# Regroupement par prêt
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index=True, left_on='SK_ID_BUREAU', how='outer')
# Merge pour inclure le SK_ID_CURR
bureau_by_loan = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_by_loan, on='SK_ID_BUREAU', how='left')
# Aggrégation des stats pour chaque  client
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns=['SK_ID_BUREAU']), group_var='SK_ID_CURR', df_name='client')


In [12]:
# Fusion des résultats avec les jeux de données d'entraînement et de test
def merge_with_main(df_main, df_to_merge):
    df_main = df_main.merge(df_to_merge, on='SK_ID_CURR', how='left')
    return df_main


In [13]:
train = merge_with_main(train, bureau_counts)
train = merge_with_main(train, bureau_agg)
train = merge_with_main(train, bureau_balance_by_client)

In [14]:
test = merge_with_main(test, bureau_counts)
test = merge_with_main(test, bureau_agg)
test = merge_with_main(test, bureau_balance_by_client)

In [15]:
train.shape, test.shape

((307511, 320), (48744, 319))

## 5.4 - FE sur le dataframe 'previous_application'

Création d'une fonction pour convertir les types de données : Cela permet de réduire l'utilisation de la mémoire en utilisant des types plus efficaces pour les variables. Par exemple, `category` est souvent un meilleur type que `object'.

In [16]:
def return_size(df):
    """Retourne la taille du dataframe en gigaoctets"""
    return round(sys.getsizeof(df) / 1e9, 2)

In [17]:
def convert_types(df, print_info=False):
    """Convertit les types de données pour optimiser l'utilisation de la mémoire."""
    original_memory = df.memory_usage().sum()
    
    for c in df:
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Utilisation de la mémoire originale : {round(original_memory / 1e9, 2)} gb.')
        print(f'Nouvelle utilisation de la mémoire : {round(new_memory / 1e9, 2)} gb.')
        
    return df


On peut maintenant attaquer le dataframe **previous_application**

Il s'agit des demandes de crédit antérieures auprès de Home Credit. Chaque prêt en cours dans les données de la demande peut avoir plusieurs prêts antérieurs. Chaque demande antérieure a une ligne et est identifiée par la caractéristique `SK_ID_PREV`. 

In [18]:
# Lecture du fichier 'previous_application' et conversion des types
previous = pd.read_csv('previous_applications.csv')
previous = convert_types(previous, print_info=True)


Utilisation de la mémoire originale : 0.49 gb.
Nouvelle utilisation de la mémoire : 0.18 gb.


In [19]:
# Calcul des statistiques pour 'previous_application'
previous_agg = agg_numeric(previous, group_var='SK_ID_CURR', df_name='previous')
previous_counts = agg_categorical(previous, parent_var='SK_ID_CURR', df_name='previous')


In [20]:
# Fusion des résultats avec les jeux de données d'entraînement et de test
train = merge_with_main(train, previous_counts)
train = merge_with_main(train, previous_agg)

test = merge_with_main(test, previous_counts)
test = merge_with_main(test, previous_agg)

In [21]:
# Gestion de la mémoire
gc.enable()
del previous, previous_agg, previous_counts
gc.collect()

0

## 5.5 - FE sur le dataframe POS_CASH_balance

In [22]:
def aggregate_client(df, group_vars, df_names):
    """Agrège les données au niveau des prêts au niveau des clients."""
    df_agg = agg_numeric(df, group_var=group_vars[0], df_name=df_names[0])
    
    if any(df.dtypes == 'object') or any(df.dtypes == 'category'):
        df_counts = agg_categorical(df, parent_var=group_vars[0], df_name=df_names[0])
        df_by_loan = df_counts.merge(df_agg, on=group_vars[0], how='outer')

        del df_agg, df_counts
        gc.collect()

        df_by_loan = df_by_loan.merge(df[[group_vars[0], group_vars[1]]], on=group_vars[0], how='left')
        df_by_loan = df_by_loan.drop(columns=[group_vars[0]])

        df_by_client = agg_numeric(df_by_loan, group_var=group_vars[1], df_name=df_names[1])
    else:
        df_by_loan = df_agg.merge(df[[group_vars[0], group_vars[1]]], on=group_vars[0], how='left')
        del df_agg
        gc.collect()

        df_by_loan = df_by_loan.drop(columns=[group_vars[0]])
        df_by_client = agg_numeric(df_by_loan, group_var=group_vars[1], df_name=df_names[1])
        
    del df, df_by_loan
    gc.collect()

    return df_by_client

Pour mémoire :
**POS_CASH_BALANCE**: les données mensuelles sur les prêts au point de vente ou au comptant que les clients ont contractés auprès de Home Credit. Chaque ligne correspond à un mois d'un prêt au point de vente ou d'un prêt au comptant antérieur, et un seul prêt antérieur peut avoir plusieurs lignes.

In [23]:
# Lecture du fichier 'POS_CASH_balance' et conversion des types
cash = pd.read_csv('POS_CASH_balances.csv')
cash = convert_types(cash, print_info=True)

Utilisation de la mémoire originale : 0.64 gb.
Nouvelle utilisation de la mémoire : 0.41 gb.


In [24]:
# Agrégation des données 'POS_CASH_balance' au niveau des clients
cash_by_client = aggregate_client(cash, group_vars=['SK_ID_PREV', 'SK_ID_CURR'], df_names=['cash', 'client'])

In [25]:
# Fusion des résultats avec les jeux de données d'entraînement et de test
train = merge_with_main(train, cash_by_client)
test = merge_with_main(test, cash_by_client)

In [26]:
# Gestion de la mémoire
del cash, cash_by_client
gc.collect()

0

## 5.6 - FE sur le dataframe 'credit_card_balances'

Pour mémoire :
**credit_card_balance**:  données mensuelles sur les cartes de crédit que les clients ont eues précédemment avec Home Credit. Chaque ligne correspond à un mois de solde de carte de crédit, et une seule carte de crédit peut avoir plusieurs lignes.

In [27]:
# Lecture du fichier 'credit_card_balances' et conversion des types
credit = pd.read_csv('credit_card_balances.csv')
credit = convert_types(credit, print_info=True)

Utilisation de la mémoire originale : 0.71 gb.
Nouvelle utilisation de la mémoire : 0.42 gb.


In [28]:
# Agrégation des données 'credit_card_balances' au niveau des clients
credit_by_client = aggregate_client(credit, group_vars=['SK_ID_PREV', 'SK_ID_CURR'], df_names=['credit', 'client'])


In [29]:
# Fusion des résultats avec les jeux de données d'entraînement et de test
train = merge_with_main(train, credit_by_client)
test = merge_with_main(test, credit_by_client)


In [30]:
# Gestion de la mémoire
del credit, credit_by_client
gc.collect()

0

## 5.7 - FE sur le dataframe 'Installment Payments'


Pour mémoire : 
**installments_payment** l'historique des paiements pour les prêts précédents chez Home Credit. Il y a une ligne pour chaque paiement effectué et une ligne pour chaque paiement manqué.

In [31]:
# Lecture du fichier 'installments_payment' et conversion des types
installments = pd.read_csv('installments_payment.csv')
installments = convert_types(installments, print_info=True)


Utilisation de la mémoire originale : 0.87 gb.
Nouvelle utilisation de la mémoire : 0.49 gb.


In [32]:
# Agrégation des données 'installments_payment' au niveau des clients
installments_by_client = aggregate_client(installments, group_vars=['SK_ID_PREV', 'SK_ID_CURR'], df_names=['installments', 'client'])


In [33]:
# Fusion des résultats avec les jeux de données d'entraînement et de test
train = merge_with_main(train, installments_by_client)
test = merge_with_main(test, installments_by_client)


In [34]:
# Gestion de la mémoire
del installments, installments_by_client
gc.collect()


0

In [37]:
print('Final Training Shape: ', train.shape)
print('Final Testing Shape: ', test.shape)

print(f'Final training size: {return_size(train)}')
print(f'Final testing size: {return_size(test)}')


Final Training Shape:  (307511, 1329)
Final Testing Shape:  (48744, 1328)
Final training size: 3.02
Final testing size: 0.48


In [38]:
train.to_csv('train_after_fe.csv', index = False, chunksize = 500)
test.to_csv('test_after_fe.csv', index = False, chunksize = 500)